In [1]:
import os
import gc
import re
import gzip
import pandas as pd

from sklearn.model_selection import train_test_split

In [2]:
data_dir = '../data/'

In [3]:
!wget -P '../data/' https://github.com/RossiyaSegodnya/ria_news_dataset/raw/master/ria.json.gz 

--2021-03-22 05:02:11--  https://github.com/RossiyaSegodnya/ria_news_dataset/raw/master/ria.json.gz
Resolving github.com (github.com)... 140.82.121.4
Connecting to github.com (github.com)|140.82.121.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/RossiyaSegodnya/ria_news_dataset/master/ria.json.gz [following]
--2021-03-22 05:02:12--  https://media.githubusercontent.com/media/RossiyaSegodnya/ria_news_dataset/master/ria.json.gz
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1025015063 (978M) [application/octet-stream]
Saving to: ‘../data/ria.json.gz’

ria.json.gz         100%[===================>] 977.53M   294MB/s    in 3.4s    

2021-03-22 05:02:43 (285 MB/s) - ‘../data/ria.j

In [4]:
data = pd.read_json(gzip.open(os.path.join(data_dir, 'ria.json.gz')), lines=True)

In [5]:
data.head()

,text,title
0,"<p><strong></strong></p>\n<p><strong>москва, 3...","большинство детей, которых пытались увезти в с..."
1,"<p><strong></strong></p>\n<p><strong>киев, 31 ...","луценко будет работать в команде тимошенко, ес..."
2,"<p><strong></strong></p>\n<p><strong>киев, 31 ...","""лютые"" выборы: есть ли вероятность второго ма..."
3,"<p><strong></strong></p>\n<p><strong>мехико, 3...",жертвами бойни на севере мексики стали 13 моло...
4,"<p><strong></strong></p>\n<p><strong>москва, 3...",немец беррер выиграл теннисный турнир на родин...


- Уберем из заголовков и текстов символы табуляции и новой строки  
- Очистим текчты от html-тегов и прочего мусора
- В виду специфики задачи, нам достаточно только лишь начала текста для генерации заголовка, поэтому для экономии всевозможных ресурсов, будем брать только первые 1000 символов из текстов

In [6]:
data['text'] = data['text'].apply(lambda x: re.sub(r'(<.*?>)', '', x.replace('\n', ' ').replace('\t', ' ').replace('\r', ' ')))
data['text'] = data['text'].apply(lambda x: re.sub(r'(&.*?;)', '', x).strip()[:1000])
data['title'] = data['title'].apply(lambda x: x.replace('\n', ' ').replace('\t', ' ').replace('\r', ' '))

- Уберем сэмплы с пустыми строками

In [7]:
data = data[(data.text != '') & (data.title != '')]

- Из-за недостатка мощностей, возьмем только лишь ~2/3 датасета для обучения

In [8]:
data = data.sample(615000, random_state=10)

Разделим данные на train, val, test:
 - train 600k сэмплов
 - val 5k сэмплов
 - test 10k сэмплов

In [9]:
train, test = train_test_split(data, test_size=10000, random_state=10)
train, val = train_test_split(train, test_size=5000, random_state=10)

In [10]:
for dataset in [train, val, test]:
    dataset.reset_index(drop=True, inplace=True)

In [11]:
train.head()

,text,title
0,"сочи, 19 сен риа новости. втб невидит другого ...","втб собрался судиться с ""мечелом"""
1,"владивосток, 11 авг - риа новости. инаугурация...",губернатор сахалинской области официально всту...
2,"каир, 13 авг - риа новости, маргарита кислова....","профессия муэдзинов, призывающих к молитве, мо..."
3,"владивосток, 24 янв риа новости. сотрудники ги...",облава на пьяных водителей пройдет в приморье ...
4,сын ливийского лидера саиф аль-ислам каддафи о...,"сын муамара каддафи пригласил москвичей в ""пус..."


In [12]:
train.to_csv(os.path.join(data_dir, 'train.csv'), index=None, sep='\t')
val.to_csv(os.path.join(data_dir, 'val.csv'), index=None, sep='\t')
test.to_csv(os.path.join(data_dir, 'test.csv'), index=None, sep='\t')